# Coding a spam classifier with naive Bayes

### 1. Imports and pre-processing data

We load the data into a Turi Create SFrame, and then preprocess it by adding a string with the (non-repeated) words in the email.

In [1]:
import numpy as np

In [2]:
import pandas as pd
emails = pd.read_csv('./emails.csv')

In [3]:
#emails = turicreate.SFrame('./emails.csv')

In [4]:
emails[:10]

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


In [5]:
def process_email(text):
    return list(set(text.split()))

emails['words'] = emails['text'].apply(process_email)

In [7]:
emails[:10]

,text,spam,words
0,Subject: naturally irresistible your corporate...,1,"[change, changes, isoverwhelminq, have, should..."
1,Subject: the stock trading gunslinger fanny i...,1,"[not, pirogue, attainder, is, clothesman, ramb..."
2,Subject: unbelievable new homes made easy im ...,1,"[-, our, have, ask, of, post, home, is, ,, new..."
3,Subject: 4 color printing special request add...,1,"[-, our, here, of, information, goldengraphix,..."
4,"Subject: do not have money , get software cds ...",1,"[not, have, here, is, ?, ,, !, ended, me, comp..."
5,"Subject: great nnews hello , welcome to medzo...",1,"[-, have, of, one, blister, 75, is, confidant,..."
6,Subject: here ' s a hot play in motion homela...,1,"[have, kind, detect, might, lightning, enabie,..."
7,Subject: save your money buy getting this thin...,1,"[not, -, thing, than, here, have, of, country,..."
8,Subject: undeliverable : home based business f...,1,"[2001, p, reach, home, us, a, ptt, based, ims,..."
9,Subject: save your money buy getting this thin...,1,"[not, -, thing, than, here, have, of, country,..."


In [8]:
# Calculating the probability that an email is spam
1.0*sum(emails['spam']==1)/len(emails)

0.2388268156424581

### 2. Coding Naive Bayes

We start by counting how many spam and ham emails contain a given word.

We check for the words 'money' and 'easy'.

In [56]:
def count_spam_ham(word):
    email_count = {'spam': 0, 'ham': 0}
    for index, email in emails.iterrows():
        if word in email['words']:
            if email['spam']:
                email_count['spam'] += 1
            else:
                email_count['ham'] += 1
    return email_count

# In case it's a dictionary
'''
def count_spam_ham(word):
    email_count = {'spam': 0, 'ham': 0}
    for email in emails:
        if word in email['word_count']:
            if email['spam']:
                email_count['spam'] += 1
            else:
                email_count['ham'] += 1
    return email_count
'''

"\ndef count_spam_ham(word):\n    email_count = {'spam': 0, 'ham': 0}\n    for email in emails:\n        if word in email['word_count']:\n            if email['spam']:\n                email_count['spam'] += 1\n            else:\n                email_count['ham'] += 1\n    return email_count\n"

In [57]:
print(count_spam_ham('money'))
print(count_spam_ham('easy'))

{'spam': 280, 'ham': 87}
{'spam': 110, 'ham': 61}


Now we make a function that takes a number of words. The naive Bayes algorithm goes over all these words, multiplies the probabilities that the email containing them are spam, and ham. Finally, calculates the weighted probabilities using Naive Bayes, and returns the probability that the email is spam.

In [58]:
def prob_spam_bayes(word):
    # Returns the probability that the email is spam given that it contains a word
    spam, ham = count_spam_ham(word)
    if spam==0 and ham==0:
        return 0.5
    return 1.0*spam/(spam+ham)

In [69]:
def prob_spam_naive_bayes(words):
    email_counts = [count_spam_ham(word) for word in words]
    spams = [count['spam'] for count in email_counts]
    hams = [count['ham'] for count in email_counts]
    #print spams
    #print hams
    spam = np.prod([count['spam'] for count in email_counts])
    ham = np.prod([count['ham'] for count in email_counts])
    if spam==0 and ham==0:
        return 0.5
    return 1.0*spam/(spam+ham)

# In case the email comes as a string
def prob_spam_naive_bayes_string(email):
    words = email.split()
    return prob_spam_naive_bayes(words)

### Testing with some sample emails
We verify that for non-spammy words, the classifier gives us small probabilities, and for spammy words it gives us large probabilities.

In [70]:
prob_spam_naive_bayes(['money', 'easy'])

0.8530201899908605

In [71]:
prob_spam_naive_bayes(['mom','friend','school'])

0.008857887217413228

In [72]:
prob_spam_naive_bayes(['prince','viagra'])

1.0

In [73]:
prob_spam_naive_bayes_string('hi mom how are you please buy apples')

0.0

In [74]:
prob_spam_naive_bayes_string('buy cheap viagra get lottery')

1.0

In [75]:
prob_spam_naive_bayes_string('enter in the lottery now win three million dollars')

1.0

In [76]:
prob_spam_naive_bayes_string('lets meet at the hotel lobby at nine am tomorrow')

0.0

In [77]:
prob_spam_naive_bayes_string('hi mom make easy money')

0.08279582746750283

In [78]:
prob_spam_naive_bayes_string('hi mom')

0.03860711582134747

In [79]:
prob_spam_naive_bayes_string('make easy money')

0.6921082499793675

### 3. Training an efficient model

Our plan is to write a dictionary, and in this dictionary record every word, and its pair of occurrences in spam and ham

In [84]:
model = {}

# Training process
for index, email in emails.iterrows():
    for word in email['words']:
        if word not in model:
            model[word] = {'spam': 1, 'ham': 1}
        if word in model:
            if email['spam']:
                model[word]['spam'] += 1
            else:
                model[word]['ham'] += 1

In [85]:
model

{':': {'spam': 798, 'ham': 3639},
 'shouldn': {'spam': 28, 'ham': 9},
 'world': {'spam': 122, 'ham': 161},
 'but': {'spam': 251, 'ham': 1016},
 'that': {'spam': 533, 'ham': 2471},
 'see': {'spam': 261, 'ham': 813},
 'guaranteed': {'spam': 96, 'ham': 15},
 'creativeness': {'spam': 25, 'ham': 1},
 'formats': {'spam': 50, 'ham': 8},
 'organization': {'spam': 42, 'ham': 140},
 'of': {'spam': 973, 'ham': 3326},
 'stylish': {'spam': 15, 'ham': 1},
 'we': {'spam': 677, 'ham': 2378},
 'use': {'spam': 207, 'ham': 540},
 'interested': {'spam': 186, 'ham': 457},
 'done': {'spam': 81, 'ham': 229},
 'corporate': {'spam': 49, 'ham': 193},
 'list': {'spam': 243, 'ham': 400},
 'easy': {'spam': 111, 'ham': 62},
 'structure': {'spam': 31, 'ham': 129},
 'to': {'spam': 1162, 'ham': 4057},
 'naturally': {'spam': 9, 'ham': 8},
 'system': {'spam': 115, 'ham': 344},
 'satisfaction': {'spam': 63, 'ham': 7},
 'look': {'spam': 140, 'ham': 786},
 'ieader': {'spam': 17, 'ham': 1},
 'three': {'spam': 89, 'ham': 192

In [86]:
model['lottery']

{'spam': 9, 'ham': 1}

In [87]:
model['sale']

{'spam': 39, 'ham': 42}

In [88]:
def predict_bayes(word):
    num_spam_with_word = model[word]['spam']
    num_ham_with_word = model[word]['ham']
    return 1.0*num_spam_with_word/(num_spam_with_word + num_ham_with_word)

In [89]:
predict_bayes('lottery')

0.9

In [90]:
predict_bayes('sale')

0.48148148148148145

In [93]:
def predict_naive_bayes(email):
    words = set(email.split())
    spams = []
    hams = []
    for word in words:
        if word in model:
            spams.append(model[word]['spam'])
            hams.append(model[word]['ham'])
    prod_spams = np.long(np.prod(spams))
    prod_hams = np.long(np.prod(hams))
    return 1.0*prod_spams/(prod_spams + prod_hams)

In [94]:
predict_naive_bayes('hi mom how are you')

0.0013894756610580057

In [95]:
predict_naive_bayes('enter the lottery to win three million dollars')

0.38569290647197135

In [96]:
predict_naive_bayes('meet me at the lobby of the hotel at nine am')

0.02490194297492509

In [97]:
predict_naive_bayes('buy cheap lottery easy money now')

0.9913514898646872